In [44]:
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer, Metadata, Interpreter

In [45]:
def train_nlu(data, configuration, model_dir):
    training_data = load_data(data)
    trainer = Trainer(config.load(configuration))
    trainer.train(training_data)
    model_directory = trainer.persist(model_dir,fixed_model_name='faq_chatbot_nlu')

In [46]:
train_nlu_model_switch = True

In [65]:
if (train_nlu_model_switch):
    train_nlu('./data/data.json','spacy_configuration.json','./models/faq_chatbot_nlu')

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.3s finished


In [66]:
def run_nlu(query):
    interpreter = Interpreter.load('./models/faq_chatbot_nlu/default/faq_chatbot_nlu')
    return interpreter.parse(query)

In [75]:
query = 'can you explain the meaning of usd'

In [76]:
parsed_query = run_nlu(query)
parsed_query

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/faq_chatbot_nlu/default/faq_chatbot_nlu\entity_synonyms.json'
  "".format(entity_synonyms_file))


{'entities': [{'confidence': 0.5908090884932593,
   'end': 34,
   'entity': 'area',
   'extractor': 'ner_crf',
   'start': 31,
   'value': 'usd'}],
 'intent': {'confidence': 0.87652434870120977, 'name': 'definition'},
 'intent_ranking': [{'confidence': 0.87652434870120977, 'name': 'definition'},
  {'confidence': 0.047450152623332527, 'name': 'address'},
  {'confidence': 0.015922747847979209, 'name': 'conditions'},
  {'confidence': 0.015631673336571162, 'name': 'contact'},
  {'confidence': 0.015554067624025126, 'name': 'benefits'},
  {'confidence': 0.013287554924850311, 'name': 'goodbye'},
  {'confidence': 0.0090449040013226324, 'name': 'greet'},
  {'confidence': 0.0065845509407090972, 'name': 'gratitude'}],
 'text': 'can you explain the meaning of usd'}